# Let's forget about `net-tools` and welcome `iproute2`

`iproute2` is a bunch of tools that aim to replace the old commands like `ifconfig`, `route` or `arp` contained in the `net-tools` package. A lot of people are still using `net-tools` but as described in [this official statement](https://lists.debian.org/debian-devel/2009/03/msg00780.html), `net-tools` is intended to deprecation and does not support some new kernel network functionalities. Some linux distribution like Archlinux already [deprecated the `net-tools` utility](https://www.archlinux.org/news/deprecation-of-net-tools/) but still [provide them in their repository](https://www.archlinux.org/packages/core/i686/net-tools/) for backward compatibility scripts.

Beside the lack of functionalities, there is also a lack of efficiency in the `net-tools` package. `net-tools` commands reads informations from the `/proc` directory while `iproute2` uses the Linux kernel interface [Netlink](https://en.wikipedia.org/wiki/Netlink) which is much faster.

## An example of `iproute2` strenght versus `net-tools`

Let's compare the `net-tools`' `route` utility and the `ip route` utility. In most of the cases, classic routing is enough for what you want to do. For example if you want to modify the default route, both of `iproute2` and `net-tools` embed this functionality :

```bash
# net-tools version :
route add default gw $ip

# iproute2 version :
ip route add default via $ip
```

However sometimes you do not want to route your packet only based on the destination adresse of a packet. As said in the [`ip rule` manual](http://man7.org/linux/man-pages/man8/ip-rule.8.html) :

>In some circumstances we want to route packets differently depending not only on destination addresses, but also on other packet fields: source address, IP protocol, transport protocol ports or even packet payload. This task is called *'policy routing'*.

### Example of situation that requires policy routing

Let's say we have the following machine :

```
+----- Machine ----+      VPN subnet       +-- Router --+
| 'tun0' interface  >--- 10.10.10.0/24 ---<  10.10.10.1 |
|       10.10.10.x |                       +------------+
|                  |
|                  |      WAN subnet       +-- Router --+
|  'wan' interface  >--- 172.17.0.0/24 ---<  172.17.0.1 |
|      172.17.0.2  |                       +------------+
+------------------+
```

This machine is subject to these constraints :

- the default route of every packets is over the VPN's router (i.e. `10.10.10.1`)
- **only** packets marked `32/32` (VPN packets and some other packets) must go through the `wan` interface (`32/32` is an example).

In this situation, the problem is the last constraint. Using the classic routing which is the purpose of `net-tools`' `route` utility, you cannot associate a mark to a specific route.

A solution (there are tons of different solutions) is to create a new routing table `wan-force` in the `/etc/iproute2/rt_table` :

```
#
# reserved values
#
255 local
254 main
253 default
0   unspec
#
# local
#
1 wan-force
```

Then bind every packet marked `32/32` to the `wan-force` table using `ip rule` :

```bash
ip rule fwmark "32/32" priority 16286 lookup wan-force
```

If now you list your rules using `ip rule list`, you will see the different rules and their priority :

```
0:      from all lookup local 
16286:  from all fwmark 0x20/0x20 lookup wan-force 
32766:  from all lookup main 
32767:  from all lookup default 
```

Finally add the default routes in the `wan-force` and the main table

```bash
ip route add default via "172.17.0.1" table wan-force
ip route add default via "10.10.10.1" # table main is automatically selected
```

Now every packet marked `32/32` will be automatically routed through the `wan` interface.

# `net-tools` and `iproute2` commands cheatsheet

It is sometimes hard to get into `iproute2` when you are used to use the `net-tools` commands. In order to make it easier, here is a small cheatsheet from the [Red Hat Entreprise `ip` command cheatsheet](https://access.redhat.com/sites/default/files/attachments/rh_ip_command_cheatsheet_1214_jcs_print.pdf).

<table>
    <tr>
        <th>`net-tools` command</th>
        <th>`iproute2` command</th>
        <th>Command purpose</th>
    </tr>
    <tr>   
        <td>`arp -na`</td>
        <td>`ip neigh`</td>
        <td>Display the neighbour objects (i.e. the arp table)</td>
    </tr>
    <tr>
        <td>`ifconfig`</td>
        <td>`ip link`</td>
        <td>Manage and display the state of all network interfaces</td>
    </tr>
    <tr>
        <td>`ifconfig -a`</td>
        <td>`ip addr show`</td>
        <td>Display IP addresses and propertty information</td>
    </tr>
    <tr>
        <td>`ifconfig -s`</td>
        <td>`ip -s link`</td>
        <td>Display the network statistics per interfaces</td>
    </tr>
    <tr>
        <td>`ifconfig eth0 up|down`</td>
        <td>`ip link set eth0 up|down`</td>
        <td>Enable / disable an interface</td>
    </tr>
    <tr>
        <td>`netstat`</td>
        <td>`ss`</td>
        <td>Display socket statistics (see the Red Hat cheatsheet for some useful options)</td>
    </tr>
    <tr>
        <td>`route [...]`</td>
        <td>`ip route [...] [table {table-name}]`</td>
        <td>Display and alter the routing table(s)</td>
    </tr>
    <tr>
        <td></td>
        <td>`ip rule`</td>
        <td>Display and manage the rules for routing table affectation</td>
    </tr>
</table>